In [2]:
import sqlite3
conn = sqlite3.connect('factbook.db')
schema = conn.execute("pragma table_info(facts);").fetchall()
for s in schema:
    print(s)

(0, 'id', 'INTEGER', 1, None, 1)
(1, 'code', 'varchar(255)', 1, None, 0)
(2, 'name', 'varchar(255)', 1, None, 0)
(3, 'area', 'integer', 0, None, 0)
(4, 'area_land', 'integer', 0, None, 0)
(5, 'area_water', 'integer', 0, None, 0)
(6, 'population', 'integer', 0, None, 0)
(7, 'population_growth', 'float', 0, None, 0)
(8, 'birth_rate', 'float', 0, None, 0)
(9, 'death_rate', 'float', 0, None, 0)
(10, 'migration_rate', 'float', 0, None, 0)


### Viewing Query Plans to check for efficiency

In [3]:
conn.execute("CREATE INDEX IF NOT EXISTS name_idx ON facts(name)")

In [4]:
query_plan_one = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE area > 40000").fetchall()
print(query_plan_one)
query_plan_two = conn.execute("EXPLAIN QUERY PLAN SELECT area FROM facts WHERE area > 40000").fetchall()
print(query_plan_two)
query_plan_three = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE name = 'Czech Republic'").fetchall()
print(query_plan_three)
query_plan_four = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE id=20").fetchall()
print(query_plan_four)


[(0, 0, 0, 'SCAN TABLE facts')]
[(0, 0, 0, 'SCAN TABLE facts')]
[(0, 0, 0, 'SEARCH TABLE facts USING INDEX name_idx (name=?)')]
[(0, 0, 0, 'SEARCH TABLE facts USING INTEGER PRIMARY KEY (rowid=?)')]


More query plans. You may wonder where query_plan_five went. I wonder the same thing. 

In [5]:
query_plan_six = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 10000").fetchall()
print(query_plan_six)
conn.execute("CREATE INDEX IF NOT EXISTS pop_idx ON facts(population)")
query_plan_seven = conn.execute("EXPLAIN QUERY PLAN SELECT * FROM facts WHERE population > 10000").fetchall()
print(query_plan_seven)

[(0, 0, 0, 'SCAN TABLE facts')]
[(0, 0, 0, 'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


Knowing the query plans will allow for increased efficiency when searching databases. At least, it will allow for some knowledge of what needs to be optimized. If the query is searching the entire table, perhaps there should be an index addressing the specific column that we are searching.